In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-274781
Azure region: southcentralus
Subscription id: aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee
Resource group: aml-quickstarts-274781


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "optmizing-ml"
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
### YOUR CODE HERE ###
compute_config = AmlCompute.provisioning_configuration(
    vm_size="Standard_D2_V2",
    max_nodes=4
)

compute_target = ComputeTarget(workspace=ws,name= cluster_name)

In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C': uniform(0.1,1),
        '--max_iter':choice(50,100)
    }
)

# Specify a Policy
policy = BanditPolicy(
    evaluation_interval=1,
    slack_factor=0.1
)

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='./training/conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(
    source_directory="./",
    script="train.py",
    compute_target=compute_target,
    environment=sklearn_env
)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name="accuracy",
    primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=10,
    max_concurrent_runs=4
)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hdr = exp.submit(hyperdrive_config)  # Submete o job para execução
hdr.wait_for_completion(show_output=True)  # Espera a conclusão do job
assert(hdr.get_status() == "Completed")

RunId: HD_c8bec809-9fd0-4eb3-8d55-9d93d4ad2c6a
Web View: https://ml.azure.com/runs/HD_c8bec809-9fd0-4eb3-8d55-9d93d4ad2c6a?wsid=/subscriptions/aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee/resourcegroups/aml-quickstarts-274781/workspaces/quick-starts-ws-274781&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2025-03-04T20:23:57.2518845Z][GENERATOR][DEBUG]Sampled 4 jobs from search space 
[2025-03-04T20:23:57.4444436Z][SCHEDULER][INFO]Scheduling job, id='HD_c8bec809-9fd0-4eb3-8d55-9d93d4ad2c6a_0' 
[2025-03-04T20:23:57.5852591Z][SCHEDULER][INFO]Scheduling job, id='HD_c8bec809-9fd0-4eb3-8d55-9d93d4ad2c6a_3' 
[2025-03-04T20:23:57.5861311Z][SCHEDULER][INFO]Scheduling job, id='HD_c8bec809-9fd0-4eb3-8d55-9d93d4ad2c6a_2' 
[2025-03-04T20:23:57.4730377Z][SCHEDULER][INFO]Scheduling job, id='HD_c8bec809-9fd0-4eb3-8d55-9d93d4ad2c6a_1' 
[2025-03-04T20:23:57.9705455Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_c8bec809-9fd0-4eb3-8d55-9d93d4ad2c6a_0' 
[2025-03-04T20:2

In [5]:
import joblib
# Get your best run and save the model from that run.
best_run = hdr.get_best_run_by_primary_metric()


In [6]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,HD_c8bec809-9fd0-4eb3-8d55-9d93d4ad2c6a_1,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [7]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"  # Substitua pela URL real
ds = TabularDatasetFactory.from_delimited_files(path=url)

In [35]:
from train import clean_data
from azureml.core import Dataset
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
cleaned_data = pd.concat([x,y],axis=1)
cleaned_data.columns = cleaned_data.columns.str.replace('.','_')

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


In [38]:
cleaned_ds = Dataset.Tabular.register_pandas_dataframe(
    dataframe=cleaned_data,
    target= ws.get_default_datastore(),
    name='cleaned_bankmarketing_data')

Validating arguments.
Arguments validated.
Validating arguments.
Arguments validated.
'overwrite' is set to True. Any file already present in the target will be overwritten.
Uploading files from '/tmp/tmpqu8284g8' to 'managed-dataset/b280ba9d-b90a-4f00-b9fb-fb415aa73b55/'
Copying 1 files with concurrency set to 1
Copied /tmp/tmpqu8284g8/dataframe.parquet, file 1 out of 1. Destination path: https://mlstrg274781.blob.core.windows.net/azureml-blobstore-0c2b9052-43b9-47d0-aeff-11351b958553/managed-dataset/b280ba9d-b90a-4f00-b9fb-fb415aa73b55/dataframe.parquet
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Files copied=1, skipped=0, failed=0
Successfully created and registered a new dataset.


In [39]:
cleaned_ds = Dataset.get_by_name(ws, name='cleaned_bankmarketing_data')
print(cleaned_ds)

TabularDataset
{
  "definition": "EnginelessDataflow:\n---\ntype: mltable\npaths:\n  - pattern: \"azureml://subscriptions/aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee/resourcegroups/aml-quickstarts-274781/workspaces/quick-starts-ws-274781/datastores/workspaceblobstore/paths/managed-dataset/b280ba9d-b90a-4f00-b9fb-fb415aa73b55/\"\ntransformations:\n  - read_parquet:\n      path_column: Path\n      include_path_column: false\nmetadata:\n  infer_column_types: \"False\"\n",
  "registration": {
    "id": "59a4dd56-0507-46cf-97bb-6f97066f1e46",
    "name": "cleaned_bankmarketing_data",
    "version": 3,
    "workspace": "Workspace.create(name='quick-starts-ws-274781', subscription_id='aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee', resource_group='aml-quickstarts-274781')"
  }
}


In [40]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=cleaned_ds,
    label_column_name='y',
    n_cross_validations=2,
    compute_target= compute_target)

In [ ]:
# Submit your automl run

### YOUR CODE HERE ###
remote_run = exp.submit(automl_config, show_output =True)
remote_run.wait_for_completion()

Submitting remote run.
No run_configuration provided, running on optmizing-ml with default configuration
Running on remote compute: optmizing-ml


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_f241d7cc-95e3-4e3d-9228-c00039e920ed,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+-------------------------------------

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run, fitted_model = remote_run.get_output()